<a href="https://colab.research.google.com/github/JoshZastrow/Google-Analytics/blob/master/load_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

## Clone the Project Repo

In [0]:
!git config --global user.email "j.a.zastrow.jr@gmail.com"
!git config --global user.name "JoshZastrow"

In [0]:
!git clone https://github.com/acclimate-consulting/Google-Analytics.git 

## Load Data

Create API Token from **www.kaggle.com/<your_username>/account** and download the API token. Run the code below to upload the key and set it as a local variable.


In [49]:
from google.colab import files
import os
import json

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    
    with open(fn) as json_data:
        API_KEY = json.load(json_data)

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes


####Store API Keys

The Kaggle API Key should be loaded in as a variable `API_KEY`, but you probably won't need to use this. The kaggle.json API file is in the directory, so we're going to move it to the right config location.

In [0]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!rm kaggle.json

#### Download Kaggle package and fetch data

In [0]:
!pip install kaggle
!kaggle competitions download -c ga-customer-revenue-prediction

#### Unzip files

In [0]:
!unzip sample_submission_v2.csv.zip 

In [0]:
!unzip train_v2.csv.zip

In [0]:
!unzip test_v2.csv.zip

In [70]:
import os

'train file is {:2f} gb'.format(os.path.getsize("train_v2.csv")*0.000000001)

'train file is 25.412028 gb'

## Data Format (From Kaggle)
Both train_v2.csv and test_v2.csv contain the columns listed under Data Fields. Each row in the dataset is one visit to the store. Because we are predicting the log of the total revenue per user, be aware that not all rows in test_v2.csv will correspond to a row in the submission, but all unique fullVisitorIds will correspond to a row in the submission.

IMPORTANT: Due to the formatting of fullVisitorId you must load the Id's as strings in order for all Id's to be properly unique!
There are multiple columns which contain JSON blobs of varying depth. In one of those JSON columns, totals, the sub-column transactionRevenue contains the revenue information we are trying to predict. This sub-column exists only for the training data

In [0]:
from pandas.io.json import json_normalize
import json
import os

def load_df(csv_path='test_v2.csv.zip', chunksize=20, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows,
                     chunksize=chunksize,
                     compression='zip')
    
    for chunk in df:
        for column in JSON_COLUMNS:
            column_as_df = json_normalize(chunk[column])
            
            column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
            chunk = chunk.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
        # print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
        yield chunk

In [68]:
for df_chunk in load_df():
    x = df_chunk
    break
x.head()   

,channelGrouping,customDimensions,date,fullVisitorId,hits,socialEngagementType,visitId,visitNumber,visitStartTime,device.device,geoNetwork.geoNetwork,totals.totals,trafficSource.adContent,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,"[{'index': '4', 'value': 'APAC'}]",20180511,7460955084541987166,"[{'hitNumber': '1', 'time': '0', 'hour': '21',...",Not Socially Engaged,1526099341,2,1526099341,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Souther...","{'visits': '1', 'hits': '4', 'pageviews': '3',...",(not set),not available in demo dataset,(not set),True,(not provided),organic,(not set),google
1,Direct,"[{'index': '4', 'value': 'North America'}]",20180511,460252456180441002,"[{'hitNumber': '1', 'time': '0', 'hour': '11',...",Not Socially Engaged,1526064483,166,1526064483,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","{'visits': '1', 'hits': '4', 'pageviews': '3',...",(not set),not available in demo dataset,(not set),True,(not set),(none),(not set),(direct)
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20180511,3461808543879602873,"[{'hitNumber': '1', 'time': '0', 'hour': '12',...",Not Socially Engaged,1526067157,2,1526067157,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","{'visits': '1', 'hits': '4', 'pageviews': '3',...",(not set),not available in demo dataset,(not set),True,(not provided),organic,(not set),google
3,Direct,"[{'index': '4', 'value': 'North America'}]",20180511,975129477712150630,"[{'hitNumber': '1', 'time': '0', 'hour': '23',...",Not Socially Engaged,1526107551,4,1526107551,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","{'visits': '1', 'hits': '5', 'pageviews': '4',...",(not set),not available in demo dataset,(not set),True,(not set),(none),(not set),(direct)
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20180511,8381672768065729990,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,1526060254,1,1526060254,"{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","{'visits': '1', 'hits': '5', 'pageviews': '4',...",(not set),not available in demo dataset,(not set),NaN,(not provided),organic,(not set),google


In [86]:
x['geoNetwork.geoNetwork'].apply(json.loads)

JSONDecodeError: ignored

In [80]:
x['geoNetwork.geoNetwork']

0     {'continent': 'Asia', 'subContinent': 'Souther...
1     {'continent': 'Americas', 'subContinent': 'Nor...
2     {'continent': 'Americas', 'subContinent': 'Nor...
3     {'continent': 'Americas', 'subContinent': 'Nor...
4     {'continent': 'Americas', 'subContinent': 'Nor...
5     {'continent': 'Americas', 'subContinent': 'Sou...
6     {'continent': 'Americas', 'subContinent': 'Nor...
7     {'continent': 'Americas', 'subContinent': 'Nor...
8     {'continent': 'Asia', 'subContinent': 'Eastern...
9     {'continent': 'Europe', 'subContinent': 'South...
10    {'continent': 'Americas', 'subContinent': 'Nor...
11    {'continent': 'Americas', 'subContinent': 'Nor...
12    {'continent': 'Americas', 'subContinent': 'Nor...
13    {'continent': 'Americas', 'subContinent': 'Nor...
14    {'continent': 'Asia', 'subContinent': 'Western...
15    {'continent': 'Asia', 'subContinent': 'Eastern...
16    {'continent': 'Europe', 'subContinent': 'South...
17    {'continent': 'Europe', 'subContinent': 'E